In [1]:
import pandas as pd

from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.svm import LinearSVC

In [2]:
df = pd.read_csv("./data/reviews_tabular_with_split.csv")

# --- 2) Split mantığı: VALIDATE'leri de TEST gibi kullan (yani test setine dahil et) ---
train_df = df[df["split"].str.upper() == "TRAIN"].copy()
test_df  = df[df["split"].str.upper().isin(["TEST", "VALIDATE"])].copy()

X_train, y_train = train_df["review_text"].astype(str), train_df["label"].astype(str)
X_test,  y_test  = test_df["review_text"].astype(str),  test_df["label"].astype(str)


In [3]:
model = Pipeline([
    ("tfidf", TfidfVectorizer(
        lowercase=True,
        ngram_range=(1, 2),     # unigram + bigram genelde iyi
        min_df=2,               # çok nadir geçenleri at (verin azsa 1 yap)
        max_df=0.95,
        sublinear_tf=True
    )),
    ("svm", LinearSVC(
        C=1.0,
        class_weight="balanced" # sınıflar dengesizse işe yarar
    ))
])


In [4]:

# Train
model.fit(X_train, y_train)

# Evaluate
y_pred = model.predict(X_test)
#print(classification_report(y_test, y_pred, digits=4))


In [6]:
# Tahmin örneği
samples = [
    "Mükemmel hizmet, çok memnun kaldım.",
    "Ne iyi ne kötü, idare eder.",
    "Berbat. Sorun çözülmedi."
]
for s in samples:
    print(s, "->", model.predict([s])[0])

Mükemmel hizmet, çok memnun kaldım. -> Olumlu
Ne iyi ne kötü, idare eder. -> Nötr
Berbat. Sorun çözülmedi. -> Olumsuz
